In [1]:
import backtrader as bt
import backtrader.feeds as feeds
import yfinance as yf
import pandas as pd
from MAcrossover import MAcrossover

In [8]:
def strategy(statDate: str, endDate:str, pfast=[3, 5, 7], pslow=[10, 15, 20, 30, 50]):
    # 從Yahoo Finance取得資料
    data = feeds.PandasData(dataname=yf.download('QQQ', statDate, endDate, auto_adjust=True))
    # 初始化cerebro
    cerebro = bt.Cerebro(optreturn=False)
    # feed data
    cerebro.adddata(data)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    cerebro.optstrategy(MAcrossover, pfast=pfast, pslow=pslow)

    optimized_runs = cerebro.run()
    final_results_list = []
    for run in optimized_runs:
        for strategy in run:
            value = round(strategy.broker.get_value() / 10000, 2)
            sharpe = strategy.analyzers.sharpe_ratio.get_analysis()
            final_results_list.append({
                "pfast": strategy.params.pfast,
                "pslow": strategy.params.pslow,
                "value": value,
                "sharperatio": sharpe["sharperatio"]
            })
    return pd.DataFrame(final_results_list)

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
def heatmap(data: pd.DataFrame):
    graph = {}
    for pfast in data['pfast'].unique():
        for pslow in data['pslow'].unique():
            if pslow not in graph.keys():
                graph[pslow] = {}
            graph[pslow][pfast] = data[(data['pfast']==pfast) & (data['pslow']==pslow)]['value'].values[0]
    return pd.DataFrame(graph).style.background_gradient(cmap ='viridis')

In [10]:
result1819 = strategy('2015-01-01', '2019-12-31')
heatmap(result1819)

[*********************100%***********************]  1 of 1 completed


,10,15,20,30,50
3,1.610000,1.360000,1.240000,1.470000,1.570000
5,1.430000,1.150000,1.230000,1.400000,1.490000
7,1.260000,1.150000,1.370000,1.450000,1.310000


In [11]:
result1819 = strategy('2020-01-01', '2021-12-31')
heatmap(result1819)

[*********************100%***********************]  1 of 1 completed


,10,15,20,30,50
3,1.460000,1.550000,1.590000,1.490000,1.410000
5,1.490000,1.520000,1.690000,1.350000,1.490000
7,1.700000,1.590000,1.680000,1.340000,1.440000
